Install modules

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import math

Redefine Haversine formula

In [ ]:
def CalculateDistance(LOC1: list, LOC2: list):
    """Calculate the haversine distance between two sets of latitude and longitude coordinates."""
    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = math.radians(LOC1[0]), math.radians(LOC1[1])
    lat2, lon2 = math.radians(LOC2[0]), math.radians(LOC2[1])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = (math.sin(dlat / 2) ** 2) + math.cos(lat1) * math.cos(lat2) * (math.sin(dlon / 2) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Earth's mean radius in kilometers (6371 km)
    R = 6371
    distance = R * c

    return distance

Define inputs for working example #1

In [ ]:
optimization_input = {
    'WB_percentage':0.60,
    'SupplyBoats':{
        'Boat 1':30, # this refers to water capacity
        'Boat 2':25
    },
    'Depots':{
        'Starting':{
            'SDP1':{
                'Loc':[51.96237, 4.13898],
                'Cap':3,
            },
            'SDP2':{
                'Loc':[51.89795, 4.46795],
                'Cap':3
            }
        },
    'Ending':{
            'EDP1':{
            'Loc':[51.96237, 4.13898],
            'Cap':1,
        },
        'EDP2':{
            'Loc':[51.89795, 4.46795],
            'Cap':1
        }
    }

    },
    'SupplyPoints':{
        'SP1':[51.961, 4.14023],
        'SP2':[51.89839, 4.46226],
        'SP3':[51.89084, 4.31179]
    },
    'Shipments':{
        'S1':{
            'Loc':[51.9641, 3.97697],
            'WaterAmount':6
            },
        'S2':{
            'Loc':[51.9629, 4.07149],
            'WaterAmount':7
            },
        'S3':{
            'Loc':[51.92351, 4.20735],
            'WaterAmount':12
            },
        'S4':{
            'Loc':[51.90049, 4.28357],
            'WaterAmount':4
            },
        'S5':{
            'Loc':[51.88174, 4.27193],
            'WaterAmount':8
            },
        'S6':{
            'Loc':[51.87875, 4.29812],
            'WaterAmount':3
            },
        'S7':{
            'Loc':[51.89802, 4.3507],
            'WaterAmount':5
            },
        'S8':{
            'Loc':[51.88573, 4.44271],
            'WaterAmount':8
            },
        'S9':{
            'Loc':[51.8981, 4.48526],
            'WaterAmount':4
            },
        'S10':{
            'Loc':[51.91737, 4.50551],
            'WaterAmount':12
            }
    },
}

# Derive locations in one dict
locations = {x:optimization_input['Shipments'][x]['Loc'] for x in optimization_input['Shipments']}
locations.update({x:optimization_input['SupplyPoints'][x] for x in optimization_input['SupplyPoints']})
locations.update({x:optimization_input['Depots']['Starting'][x]['Loc'] for x in optimization_input['Depots']['Starting']})
locations.update({x:optimization_input['Depots']['Ending'][x]['Loc'] for x in optimization_input['Depots']['Ending']})

locations

We'll now create the model. We'll start with our decision matrix X and our cost matrix C. 

In [ ]:
# Create model
model = gp.Model()

# Create cost matrix C
shipments = [x for x in optimization_input['Shipments']] 
sdepots = [x for x in optimization_input['Depots']['Starting']]
edepots = [x for x in optimization_input['Depots']['Ending']]
supplypoints = [x for x in optimization_input['SupplyPoints']]
indexes = shipments + sdepots + edepots + supplypoints

C = {}
for i in indexes:
    for j in indexes:
        C[i, j] = CalculateDistance(locations[i], locations[j])

C

We'll create our decision matrices X and we'll immediately specify that this matrix contains binary decision variables.

In [ ]:
# Extract supplyboats
supplyboats = [x for x in optimization_input['SupplyBoats']]
supplyboats

# Create decision variable matrix X
X = {}
for s in supplyboats:
    for i in indexes:
        for j in indexes:
            X[i, j, s] = model.addVar(vtype=gp.GRB.BINARY)
            
X

# Add X to the model
#model.addVars(X, vtype=gp.GRB.BINARY, name="X")

Now, let's define the objective function.

In [ ]:
model.setObjective(gp.quicksum(X[i,j,s] * C[i,j] for i in indexes for j in indexes for s in supplyboats),gp.GRB.MINIMIZE)

Constraint(s): each **shipment** location needs to be visited exactly once. 

In [ ]:
model.addConstrs(gp.quicksum(X[i,j,s] for s in supplyboats for i in indexes if i !=j) == 1 for j in shipments)
model.addConstrs(gp.quicksum(X[i,j,s] for s in supplyboats for j in indexes if i !=j) == 1 for i in shipments)

Constraints: ensure the continuity of the route and do not allow 2-node cycles.

In [ ]:
# Ensure continuity
model.addConstrs(gp.quicksum(X[i,h,s] for i in indexes) - gp.quicksum(X[h,j,s] for j in indexes) == 0 for s in supplyboats for h in shipments + supplypoints)

# Do not allow 1-node cycles
model.addConstrs(gp.quicksum(X[i,i,s] for i in indexes) == 0 for s in supplyboats)

# Do not allow subtours
u = {}
for i in shipments+supplypoints:
    for s in supplyboats:
        u[i, s] = model.addVar(vtype=GRB.INTEGER, lb=0, ub=len(shipments+supplypoints)-1, name=f'u_{i}_{s}')

for i in shipments+supplypoints:
    for s in supplyboats:
        model.addConstr(u[i,s] >= 1)
        model.addConstr(u[i,s] <= len(shipments+supplypoints)-1)

    for j in shipments+supplypoints:
        if i != j:
            for s in supplyboats:
                model.addConstr(u[i,s] - u[j,s] + len(shipments+supplypoints) * X[i,j,s] <= len(shipments+supplypoints) - 1)

Constraints: ensure the boats start at a starting depot and end at an ending depot. 

In [ ]:
model.addConstrs(gp.quicksum(X[i,j,s] for i in sdepots for j in indexes) == 1 for s in supplyboats)
model.addConstrs(gp.quicksum(X[i,j,s] for j in sdepots for i in indexes) == 0 for s in supplyboats)

In [ ]:
model.addConstrs(gp.quicksum(X[i,j,s] for i in edepots for j in indexes) == 0 for s in supplyboats)
model.addConstrs(gp.quicksum(X[i,j,s] for j in edepots for i in indexes) == 1 for s in supplyboats)

Constraint: make sure that each supply boat supplies at maximum 60% of the total water delivered.

In [ ]:
model.addConstrs(gp.quicksum(X[i,j,s] * optimization_input['Shipments'][i]['WaterAmount'] for i in shipments for j in indexes) <= optimization_input['WB_percentage']*sum([optimization_input['Shipments'][x]['WaterAmount'] for x in optimization_input['Shipments']]) for s in supplyboats)

Constraint: per depot, only a given capacity $DC_i$ can start and end there.  

In [ ]:
model.addConstrs(gp.quicksum(X[i,j,s] for j in indexes for s in supplyboats) <= optimization_input['Depots']['Starting'][i]['Cap'] for i in sdepots)
model.addConstrs(gp.quicksum(X[i,j,s] for i in indexes for s in supplyboats) <= optimization_input['Depots']['Ending'][j]['Cap'] for j in edepots)

Constraint: let SDP_a be EDP_a for every supply boat.

In [ ]:
model.addConstrs(gp.quicksum(X['SDP{i}'.format(i=str(i+1)),j,s] for j in indexes) == gp.quicksum(X[j,'EDP{i}'.format(i=str(i+1)),s] for j in indexes) for i in range(1, len(sdepots)) for s in supplyboats) 

Now, let's introduce a variable called Y which tracks the water tanks

In [ ]:
# Add Y variables
Y = {}
for i in indexes:
    for j in indexes:
        for s in supplyboats:
          Y[i,j,s] = model.addVar(vtype=gp.GRB.CONTINUOUS)

Y

Add constraints that enforce the desired behaviour for Y

In [ ]:
# Y cannot be more than the individual capacity of a waterboat
model.addConstrs(Y[i,j,s] <= optimization_input['SupplyBoats'][s]*X[i,j,s] for i in indexes for j in indexes for s in supplyboats)

# Y cannot be less than zero. 
model.addConstrs(Y[i,j,s] >= 0 for i in indexes for j in indexes for s in supplyboats)

# Subtract the amount delivered at point i 
model.addConstrs(gp.quicksum(Y[j,i,s] for j in indexes if i != j for s in supplyboats) - gp.quicksum(Y[i,j,s] for j in indexes if i != j for s in supplyboats) == optimization_input['Shipments'][i]['WaterAmount'] for i in shipments)

# Refill tank at supply points and depots
model.addConstrs(Y[i,j,s] == optimization_input['SupplyBoats'][s]*X[i,j,s] for i in supplypoints+sdepots+edepots for j in indexes for s in supplyboats)

Run the model.

In [ ]:
model.update()
model.optimize()
#print(model.display())

Let's extract the solution.

In [ ]:
# Routes
routes = {}
for x in supplyboats:
    routes[x] = []

for x in X:
    if X[x].X == 1:
        routes[x[2]].append([x[0],x[1]])
        
routes

And let's sort the routes. 

In [ ]:
ordered_routes = {}

for x in routes:
    ordered_routes[x] = []
    for y in routes[x]:
        # Identify start
        s = [z for z in routes[x] if z[1] == y[0]]

        if len(s) == 0:
            start = y
            ordered_routes[x].append(start[0])

    while len(ordered_routes[x]) != len(routes[x])+1:
        # Find last index
        current_length = len(ordered_routes[x])-1
        current_pos = ordered_routes[x][current_length]

        # Find next index
        next_index = [z for z in routes[x] if z[0] == current_pos][0][1]
        print(next_index)
        ordered_routes[x].append(next_index)

print("Optimal solution found. Distance: {dist} km.".format(dist=round(model.objVal,2)))      
ordered_routes

Let's extract the flow as well.

In [ ]:
flow = {x:[] for x in supplyboats}
for z in Y:
    if Y[z].X > 0:
        flow[z[2]].append([z[0],z[1],Y[z].X])

def order_flows(flows):
    ordered_flows = {}

    for boat, flow in flows.items():
        # Create a mapping from start point to flow
        flow_dict = {f[0]: f for f in flow}

        # Find the start point
        start = None
        for f in flow:
            if "SDP" in f[0]:
                start = f.copy()  # Create a copy
                break
        else:
            # Handle the case where no starting point is found
            print(f"No 'SDP' starting point found for boat {boat}.")
            continue

        ordered_flow = [start]
        while len(ordered_flow) < len(flow):
            current = ordered_flow[-1]
            next_flow = flow_dict.get(current[1])
            if next_flow:
                ordered_flow.append(next_flow)
            else:
                # Handle the case where no matching flow is found
                print(f"No matching flow found for boat {boat} after {current}.")
                break

        ordered_flows[boat] = ordered_flow

    return ordered_flows


ordered_flows = order_flows(flow)
for x in ordered_flows:
    ordered_flows[x] = [y[2] for y in ordered_flows[x]]
print(ordered_flows)

We'll save a function that can solve this model based on optimization_input in solver.py